In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import hmm
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = combinators.ParticleTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float).unsqueeze(0) * 2,
        'scale': torch.ones(1, 5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(1, 5),
        'scale': torch.ones(1, 5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(1, 5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, theta=hmm_params)

In [5]:
z0, mu, sigma, pi, pi0 = init_hmm(trace=generative)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [7]:
z_last, mu, sigma, pi, pi0 = hmm_run(trace=generative)

In [8]:
generative = generative.squeeze()

In [9]:
num_particles = 100

In [10]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    smc_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [11]:
smc_hmm = smc.smc(hmm.hmm_step, 50)

In [12]:
init_hmm = combinators.Model(hmm.init_hmm, phi=smc_hmm_params)

In [13]:
samples, elbos, inference = smc.particle_mh(num_particles, init_hmm, smc_hmm, 1000, generative, use_cuda=False)

In [14]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inference[key].value == generative[key].value).sum().to(dtype=torch.float) / inference.num_particles
    print('SMC accuracy at time %d: %f' % (t, accuracy))

SMC accuracy at time 1: 1.000000
SMC accuracy at time 2: 0.000000
SMC accuracy at time 3: 0.980000
SMC accuracy at time 4: 0.000000
SMC accuracy at time 5: 0.980000
SMC accuracy at time 6: 0.980000
SMC accuracy at time 7: 0.000000
SMC accuracy at time 8: 0.000000
SMC accuracy at time 9: 0.020000
SMC accuracy at time 10: 0.020000
SMC accuracy at time 11: 0.980000
SMC accuracy at time 12: 0.020000
SMC accuracy at time 13: 0.000000
SMC accuracy at time 14: 0.000000
SMC accuracy at time 15: 0.000000
SMC accuracy at time 16: 0.000000
SMC accuracy at time 17: 1.000000
SMC accuracy at time 18: 0.000000
SMC accuracy at time 19: 0.980000
SMC accuracy at time 20: 0.020000
SMC accuracy at time 21: 0.020000
SMC accuracy at time 22: 1.000000
SMC accuracy at time 23: 1.000000
SMC accuracy at time 24: 1.000000
SMC accuracy at time 25: 0.020000
SMC accuracy at time 26: 0.020000
SMC accuracy at time 27: 0.020000
SMC accuracy at time 28: 0.000000
SMC accuracy at time 29: 0.000000
SMC accuracy at time 30

In [15]:
elbos

tensor([-242.3281, -242.3281, -242.3281, -242.3281, -233.5923, -229.7379,
        -229.7379, -229.7379, -229.7379, -229.7379, -229.7379, -229.7379,
        -229.7379, -229.7379, -229.7379, -229.7379, -229.7379, -229.7379,
        -229.7379, -229.7379, -229.7379, -229.7379, -229.7379, -229.7379,
        -229.7379, -229.7379, -228.7594, -228.7594, -228.7594, -228.7594,
        -228.7594, -228.7594, -225.2290, -225.2290, -225.2290, -225.2290,
        -225.2290, -225.2290, -225.2290, -225.2290, -225.2290, -225.2290,
        -225.2290, -225.2290, -225.2290, -225.2290, -225.2290, -225.2290,
        -225.2290, -225.2290, -225.2290, -225.2290, -225.2290, -225.2290,
        -225.2290, -225.2290, -225.2290, -225.2290, -225.2290, -225.2290,
        -225.2290, -225.2290, -225.2290, -225.2290, -225.2290, -225.2290,
        -225.2290, -225.2290, -225.2290, -225.2290, -225.2290, -225.2290,
        -225.2290, -225.2290, -225.2290, -225.2290, -225.2290, -225.2290,
        -225.2290, -225.2290, -225.229